# Transport Planning
A firm must transport machines from production plants A, B and C to warehouses X, Y and Z. Five machines are required in X, 4 in Y and 3 in Z, whereas 8 machines are available in A, 5 in B and 3 in C. The transport costs (in euros) between sites
are provided in the table below.

| Plant/Warehouse | X  | Y  | Z  |
|-----------------|----|----|----|
| A               | 50 | 60 | 30 |
| B               | 60 | 40 | 20 |
| C               | 40 | 70 | 30 |

**a)** Formulate an integer linear programming model that minimizes transport costs.

**b)** Assume that the cost of transporting a machine from plant B increases by €10 for all the machines as of the third one; that is, the 4th, the 5th, etc. Reformulate the model in Section a) by considering this assumption.

In [9]:
import pulp
import pandas as pd
#And we will use numpy to perform array operations
import numpy as np
#We will use display and Markdown to format the output of code cells as Markdown
from IPython.display import display, Markdown

# Warehouses and dictionaries
plants = ('A', 'B', 'C')
warehouses = ('X', 'Y', 'Z')
#Transportation costs
transportation_costs = [[50, 60, 30], [60, 40, 20], [40, 70, 30]]

# Instantiate model
model = pulp.LpProblem("Transport Planning", pulp.LpMinimize)

# Demand
demand = [5, 4, 3]

# Capacities
capacities = [8, 5, 3]

variables = pulp.LpVariable.dicts("x",
                                  [(i, j) for i in plants for j in warehouses],
                                  lowBound=0,
                                  cat='Integer')

model += (
             pulp.lpSum([
                 transportation_costs[i][j] * variables[(plants[i], warehouses[j])]
                 for i in range(len(plants)) for j in range(len(warehouses))])
         ), "Transportation Cost"


# Capacity constraints
for i in range(len(plants)):
    model += pulp.lpSum([
        variables[(plants[i], warehouses[j])]
        for j in range(len(warehouses))]) <= capacities[i], plants[i]

# Demand
for j in range(len(warehouses)):
    model += pulp.lpSum([
        variables[(plants[i], warehouses[j])]
        for i in range(len(plants))]) >= demand[j], warehouses[j]

In [10]:
# Solve our problem
model.solve()
print(pulp.LpStatus[model.status])

# Solution
max_z = pulp.value(model.objective)
print(max_z)

Optimal
460.0


In [19]:
var_df = pd.DataFrame.from_dict(variables, orient="index",
                                columns = ["Variables"], dtype=object)

var_df["Solution"] = var_df["Variables"].apply(lambda item: item.varValue)
index = pd.MultiIndex.from_product([plants, warehouses], names=['Plants', 'Warehouses'])
var_df2 = pd.DataFrame(var_df["Solution"], index=index, columns = ["Solution"])
display(var_df2.unstack())

Solution          
Warehouses        X    Y    Z
Plants                       
A               2.0  0.0  2.0
B               0.0  4.0  1.0
C               3.0  0.0  0.0